In [3]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt

from sklearn.preprocessing import OneHotEncoder
from sklearn.model_selection import train_test_split
from keras.preprocessing import sequence
from keras.preprocessing.text import Tokenizer
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Embedding, LSTM, Dense, Dropout

In [14]:
from sklearn.model_selection import train_test_split
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.preprocessing import LabelEncoder
from sklearn.neural_network import MLPClassifier
import numpy as np
import pandas as pd
from sklearn.metrics import accuracy_score

In [5]:
# Load the dataset to see its structure
df = pd.read_csv(r'C:\Users\bugat\OneDrive - NTNU\Vår 2024\Tekstanalyse\Prosjektoppgave\Political_detection\new_df.csv')

In [10]:
df['Processed'].fillna('', inplace=True)


In [11]:
df

,All_text,Political Lean,Subreddit,SubredditEncoded,Processed,Dependency_Tags,pos_tags
0,"No matter who someone is, how they look like, ...",Liberal,socialism,14,matter someone look like language speak wear r...,"['ROOT', 'nsubj', 'ccomp', 'prep', 'compound',...",VERB PRON VERB ADP NOUN NOUN NOUN VERB ADJ NOU...
1,Biden speech draws 38.2 million U.S. TV viewers 0,Liberal,democrats,10,biden speech draw million tv viewer,"['compound', 'nsubj', 'ROOT', 'nummod', 'compo...",PROPN NOUN VERB NUM NOUN NOUN
2,State of the union Who watched the state of th...,Liberal,DemocraticSocialism,2,state union watch state union last night opinion,"['compound', 'nsubj', 'ROOT', 'compound', 'dob...",NOUN PROPN VERB NOUN NOUN ADJ NOUN NOUN
3,We Should Just Give Poor People Money 0,Liberal,SocialDemocracy,6,give poor people money,"['ROOT', 'amod', 'dative', 'dobj']",VERB ADJ NOUN NOUN
4,Do it for the Dew 0,Liberal,democrats,10,dew,['ROOT'],NOUN
...,...,...,...,...,...,...,...
12849,Ron Paul’s Spirited Defense of WikiLeaks & Fre...,Conservative,anarchocapitalism,8,ron paul spirited defense wikileaks free infor...,"['compound', 'nsubj', 'amod', 'compound', 'nsu...",PROPN PROPN VERB NOUN NOUN ADJ NOUN
12850,"“Anarcho-capitalism, in my opinion, is a doctr...",Conservative,anarchocapitalism,8,anarcho capitalism opinion doctrinal system ev...,"['nmod', 'compound', 'compound', 'amod', 'nsub...",PROPN PROPN NOUN ADJ NOUN ADV VERB AUX VERB VE...
12851,Mises Wiki is a wiki project dedicated to the ...,Conservative,anarchocapitalism,8,mises wiki wiki project dedicate advancement a...,"['ROOT', 'compound', 'compound', 'nmod', 'amod...",VERB ADJ NOUN NOUN NOUN NOUN ADJ NOUN NOUN VER...
12852,Fireman Protection Monopoly - Is This Failed C...,Conservative,anarchocapitalism,8,fireman protection monopoly failed capitalism,"['compound', 'compound', 'nsubj', 'ROOT', 'dobj']",PROPN PROPN NOUN VERB NOUN


In [20]:
from keras.models import Sequential
from keras.layers import Dense, Input

In [34]:
unique_words = set()  # Bruk et sett for å lagre unike ord

for text in df['Processed']:
    words = text.split()  # Splitt tekst i ord, antar at mellomrom er separator
    unique_words.update(words)  # Legger til ordene i settet, som automatisk fjerner duplikater

In [26]:
texts = df['Processed'].to_list()
len(texts)

12854

In [57]:
# Trinn 1: TF-IDF Vektorisering
tfidf_vectorizer = TfidfVectorizer(max_features=len(unique_words)+1) # Begrenser til de 10 000 mest frekvente ordene
X_tfidf = tfidf_vectorizer.fit_transform(df['Processed']).toarray()

# Trinn 2: Label Encoding av `Subreddit`
labels = df['SubredditEncoded'].values
labels = labels.reshape(-1, 1)
one_hot_encoder = OneHotEncoder(sparse=False)
all_labels = one_hot_encoder.fit_transform(labels)

# Trinn 3: Tren-Test Split
X_train, X_test, y_train, y_test = train_test_split(X_tfidf, all_labels, test_size=0.2, random_state=42)

# from tensorflow.keras.utils import to_categorical

# y_train_encoded = to_categorical(y_train, 15)
# y_test_encoded = to_categorical(y_test, 15)

# Opprette modellen
model = Sequential()

# Legg til Dense-lag
model.add(Dense(120, input_shape=(X_train.shape[1],), activation='relu'))
model.add(Dropout(0.5))  # Dropout for å redusere overtilpasning
model.add(Dense(32, activation='relu'))
model.add(Dropout(0.3))
# Legg til output-laget. Antall enheter skal matche antall unike klasser
model.add(Dense(15, activation='softmax'))

# Kompilere modellen
model.compile(optimizer='adam', loss='categorical_crossentropy', metrics=['accuracy'])

# Model summary for å se detaljer om modellarkitekturen
model.summary()

c:\Users\bugat\anaconda3\Lib\site-packages\sklearn\preprocessing\_encoders.py:972: FutureWarning: `sparse` was renamed to `sparse_output` in version 1.2 and will be removed in 1.4. `sparse_output` is ignored unless you leave `sparse` to its default value.
  warnings.warn(


Model: "sequential_16"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 dense_39 (Dense)            (None, 120)               2683560   
                                                                 
 dropout_25 (Dropout)        (None, 120)               0         
                                                                 
 dense_40 (Dense)            (None, 32)                3872      
                                                                 
 dropout_26 (Dropout)        (None, 32)                0         
                                                                 
 dense_41 (Dense)            (None, 15)                495       
                                                                 
Total params: 2687927 (10.25 MB)
Trainable params: 2687927 (10.25 MB)
Non-trainable params: 0 (0.00 Byte)
_________________________________________________________________


In [58]:
history = model.fit(X_train, y_train,
                    epochs=12, # Adjust based on your needs
                    validation_data=(X_test, y_test),
                    batch_size=64)

Epoch 1/12
161/161 [==============================] - 6s 31ms/step - loss: 2.6316 - accuracy: 0.1168 - val_loss: 2.4757 - val_accuracy: 0.2520
Epoch 2/12
161/161 [==============================] - 5s 28ms/step - loss: 2.2435 - accuracy: 0.2829 - val_loss: 2.0796 - val_accuracy: 0.3329
Epoch 3/12
161/161 [==============================] - 4s 28ms/step - loss: 1.8157 - accuracy: 0.4146 - val_loss: 1.9102 - val_accuracy: 0.3839
Epoch 4/12
161/161 [==============================] - 7s 43ms/step - loss: 1.4980 - accuracy: 0.5301 - val_loss: 1.8756 - val_accuracy: 0.3862
Epoch 5/12
161/161 [==============================] - 8s 50ms/step - loss: 1.2509 - accuracy: 0.6104 - val_loss: 1.9008 - val_accuracy: 0.3917
Epoch 6/12
161/161 [==============================] - 6s 35ms/step - loss: 1.0587 - accuracy: 0.6759 - val_loss: 1.9529 - val_accuracy: 0.3855
Epoch 7/12
161/161 [==============================] - 5s 33ms/step - loss: 0.9073 - accuracy: 0.7206 - val_loss: 2.0345 - val_accuracy: 0.3796

In [ ]:
test_loss, test_acc = model.evaluate(X_test, y_test)
print(f'Test Accuracy: {test_acc}, Test Loss: {test_loss}')

In [ ]:
plt.figure(figsize=(9,7))
plt.title('Accuracy score')
plt.plot(history.history['accuracy'])
plt.plot(history.history['val_accuracy'])
plt.legend(['accuracy', 'val_accuracy'])
plt.show()
plt.figure(figsize=(9,7))
plt.title('Loss value')
plt.plot(history.history['loss'])
plt.plot(history.history['val_loss'])
plt.legend(['loss', 'val_loss'])
plt.show()